# ブレンディング（Blending）
ホールドアウト検証セットを使ってベースモデルの予測結果をメタモデルに入力する  
計算コストが低く、実装がシンプルな場合が多い

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成： y = 2*x + 1 にノイズを加える
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.1, X.shape)

# NumPy → Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size=16):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# メタモデル（スタッキング・ブレンディング用のシンプルな線形モデル）
class MetaModel(nn.Module):
    def __init__(self):
        super(MetaModel, self).__init__()
        self.linear = nn.Linear(2, 1)
        
    def forward(self, x):
        return self.linear(x)

# シンプルな学習ループ
def train_on_data(model, optimizer, X_data, y_data, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_data)
        loss = criterion(preds, y_data)
        loss.backward()
        optimizer.step()
    return loss.item()



In [11]:
# ホールドアウトセットを使ってブレンディング用データを作成
X_train, X_holdout, y_train, y_holdout = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# 基底モデル1と基底モデル2を学習（ホールドアウトセット以外で訓練）
base_model1 = SimpleMLP(hidden_size=16)
optimizer_b1 = optim.Adam(base_model1.parameters(), lr=0.01)
train_on_data(base_model1, optimizer_b1, X_train, y_train, epochs=100)

base_model2 = SimpleMLP(hidden_size=16)
optimizer_b2 = optim.Adam(base_model2.parameters(), lr=0.01)
train_on_data(base_model2, optimizer_b2, X_train, y_train, epochs=100)

# ホールドアウトセットで基底モデルの予測を取得
with torch.no_grad():
    pred1_holdout = base_model1(X_holdout)
    pred2_holdout = base_model2(X_holdout)
# スタック：ホールドアウトセットでの予測をメタモデルへの入力として利用
stacked_holdout = torch.cat([pred1_holdout, pred2_holdout], dim=1)

# メタモデルの定義と訓練（ブレンディング用）
meta_model = MetaModel()
optimizer_meta = optim.Adam(meta_model.parameters(), lr=0.01)
def train_meta(meta_model, features, target, optimizer, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        meta_model.train()
        optimizer.zero_grad()
        preds = meta_model(features)
        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
    return loss.item()

meta_loss = train_meta(meta_model, stacked_holdout, y_holdout, optimizer_meta, epochs=100)

# 最終予測：ホールドアウトセットではなく全データに対して基底モデルの予測を取得し、メタモデルに入力する
with torch.no_grad():
    pred1_all = base_model1(X_tensor)
    pred2_all = base_model2(X_tensor)
    stacked_all = torch.cat([pred1_all, pred2_all], dim=1)
    blend_preds = meta_model(stacked_all)

final_loss = nn.MSELoss()(blend_preds, y_tensor).item()
print("Blending ensemble final loss:", final_loss)


Blending ensemble final loss: 0.3350297510623932
